In [27]:
!pip install openpyxl


In [28]:
import uuid

def short_uuid():
    return uuid.uuid4().hex[:8]


In [29]:
import pandas as pd
import ast
import time as t
from datetime import datetime, timedelta

In [ ]:


df_vendeurs = pd.read_excel("vendeurs.xlsx")

# Supprimer les duplications de (id, ville)
df_unique = df_vendeurs[~df_vendeurs.apply(lambda row: row.astype(str).str.contains("UUID", case=False)).any(axis=1)]

# Liste pour stocker les clients
clients_data = []

start_time = datetime(2025, 6, 4, 8, 0, 0) 

# Boucle sur chaque combinaison unique vendeur + ville
path_id = 1
for _, row in df_unique.iterrows():
    vendeur_id = row["vendeur_id"]
    ville = row["vendeur_depart_lb"]
    zone = row["vendeur_zone_num"]

    try:
        coords = ast.literal_eval(row["vendeur_zone_carto"])
    except Exception:
        continue  # Si erreur dans les coordonnées, on saute

    try:
        points = coords[0][0]
        current_time = start_time + timedelta(minutes=30 * (path_id - 1))

        for i, (long, lat) in enumerate(points):
            client_id = short_uuid()
            clients_data.append({
                "id_vendeur": vendeur_id,
                "id_client": client_id,
                "ville": ville,
                "longitude": long,
                "latitude": lat,
                "path_id": path_id,
                "zone": zone,
                "time_stamp": current_time.strftime("%Y-%m-%d %H:%M:%S")
            })
            current_time += timedelta(minutes=1)
        path_id += 1
    except Exception:
        continue  # Si mauvaise structure

# Construction du DataFrame final
df_clients = pd.DataFrame(clients_data)

# Aperçu
print(df_clients.head(10).to_string())

# Optionnel : export CSV
df_clients.to_csv("clients.csv", index=False)


  id_vendeur id_client    ville  longitude  latitude  path_id  zone           time_stamp
0   ADBO6530  0a55c03d  Orleans    2.00469  47.18607        1    60  2025-06-04 08:00:00
1   ADBO6530  e91b7984  Orleans    2.03025  47.21715        1    60  2025-06-04 08:01:00
2   ADBO6530  5eef3567  Orleans    1.95444  47.28639        1    60  2025-06-04 08:02:00
3   ADBO6530  34b93626  Orleans    1.93905  47.28496        1    60  2025-06-04 08:03:00
4   ADBO6530  351cd7e9  Orleans    1.87295  47.29238        1    60  2025-06-04 08:04:00
5   ADBO6530  a11999a9  Orleans    1.86376  47.30760        1    60  2025-06-04 08:05:00
6   ADBO6530  7f5c2932  Orleans    1.91772  47.33589        1    60  2025-06-04 08:06:00
7   ADBO6530  6556a58e  Orleans    1.84975  47.37155        1    60  2025-06-04 08:07:00
8   ADBO6530  9fecadd2  Orleans    1.84595  47.37786        1    60  2025-06-04 08:08:00
9   ADBO6530  2992214e  Orleans    1.84058  47.40569        1    60  2025-06-04 08:09:00
